## Challenge: Data cleaning & validation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics
from scipy.stats import ttest_ind

Data Source: WELLCOME Trust between 2012 and 2013, a csv of journals, articles and their costs.
    
Questions: 
1. Determine the five most common journals and the total articles for each. 
2. Calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [2]:
df = pd.read_csv('wellcome.csv')

In [3]:
df.info

<bound method DataFrame.info of                             PMID/PMCID  \
0                                  NaN   
1                           PMC3679557   
2                23043264  PMC3506128    
3                  23438330 PMC3646402   
4                 23438216 PMC3601604    
5                           PMC3579457   
6                           PMC3709265   
7                 23057412 PMC3495574    
8                    PMCID: PMC3780468   
9                    PMCID: PMC3621575   
10                   PMCID: PMC3739413   
11                   PMCID: PMC3530961   
12                   PMCID: PMC3624797   
13                          PMC3413243   
14                          PMC3694353   
15                          PMC3572711   
16                            22610094   
17                   PMCID: PMC3586974   
18        23455506  PMCID: PMC3607399    
19          PMID: 24015914 PMC3833349    
20                       : PMC3805332    
21                                 NaN   
22

In [4]:
df.columns

Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (£) charged to Wellcome (inc VAT when charged)'],
      dtype='object')

In [5]:
#rename columns
df.rename(columns={'PMID/PMCID':'PMID_PMCID'}, inplace = True) 
df.rename(columns={'Publisher':'publisher'}, inplace = True)
df.rename(columns={'Journal title':'journal_title'}, inplace = True) 
df.rename(columns={'Article title':'Article_title'}, inplace = True) 
df.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)':'cost'}, inplace = True) 

In [6]:
df.head()

,PMID_PMCID,publisher,journal_title,Article_title,cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [7]:
df.dtypes

PMID_PMCID       object
publisher        object
journal_title    object
Article_title    object
cost             object
dtype: object

In [8]:
df.isnull().sum()/df.count()

PMID_PMCID       0.103216
publisher        0.000000
journal_title    0.000470
Article_title    0.000000
cost             0.000000
dtype: float64

In [9]:
df['journal_title'] = df['journal_title'].replace(np.nan, '', regex=True)

In [10]:
df['journal_title'].isnull().value_counts()

False    2127
Name: journal_title, dtype: int64

### 1. Determine the five most common journals and the total articles for each.

In [11]:
df['journal_title'].nunique()

985

In [12]:
df['journal_title'].value_counts()

PLoS One                                                   92
PLoS ONE                                                   62
Journal of Biological Chemistry                            48
Nucleic Acids Research                                     21
Proceedings of the National Academy of Sciences            19
Human Molecular Genetics                                   18
PLoS Neglected Tropical Diseases                           18
Nature Communications                                      17
Neuroimage                                                 15
PLoS Genetics                                              15
PLoS Pathogens                                             15
NeuroImage                                                 14
Brain                                                      14
PLOS ONE                                                   14
BMC Public Health                                          14
Movement Disorders                                         13
Biochemi

In [13]:
# Convert all strings to lower case so related titles are counted together.

df['journal_lower'] = df['journal_title'].str.lower()

In [14]:
df['journal_lower'].value_counts()

plos one                                                         190
journal of biological chemistry                                   53
neuroimage                                                        29
plos pathogens                                                    24
plos genetics                                                     24
nucleic acids research                                            23
proceedings of the national academy of sciences                   20
plos neglected tropical diseases                                  20
human molecular genetics                                          19
nature communications                                             19
bmc public health                                                 14
brain                                                             14
movement disorders                                                13
developmental cell                                                12
journal of neuroscience           

In [15]:
# create DataFrame with short journal title names or acronyms, to check if they coincide with full name titles

short = df[(df['journal_lower'].apply(lambda x: len(x)<7))]
short['journal_lower'].nunique()

33

In [16]:
short['journal_lower'].value_counts()

brain     14
bmj       10
neuron     9
plos 1     7
blood      7
cortex     6
pnas       6
cell       5
pain       4
trials     4
plos       4
aids       3
gene       3
gut        3
bone       3
mbio       2
glia       2
rna        2
thorax     2
age        2
hpb        1
chest      1
heart      1
lancet     1
ijtld      1
bjp        1
hernia     1
embo       1
pntd       1
jerph      1
acta f     1
           1
acta d     1
Name: journal_lower, dtype: int64

In [17]:
# Check whether other top journals have matching acronym entries. (returns empty set)

other_acr = df[(df['journal_lower'].isin(['jbc','nar', 'hmg']))]
other_acr

,PMID_PMCID,publisher,journal_title,Article_title,cost,journal_lower


In [18]:
# Combine all journals related to Plos One.

df['journal_lower'].replace('plosone','plos one', inplace = True)
df['journal_lower'].replace('plos 1', 'plos one', inplace = True)

In [19]:
df['journal_lower'].value_counts()

plos one                                                         206
journal of biological chemistry                                   53
neuroimage                                                        29
plos pathogens                                                    24
plos genetics                                                     24
nucleic acids research                                            23
proceedings of the national academy of sciences                   20
plos neglected tropical diseases                                  20
human molecular genetics                                          19
nature communications                                             19
brain                                                             14
bmc public health                                                 14
movement disorders                                                13
biochemical journal                                               12
journal of neuroscience           

In [20]:
df.head()

,PMID_PMCID,publisher,journal_title,Article_title,cost,journal_lower
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00,psychological medicine
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,biomacromolecules
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,j med chem
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,j med chem
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88,j org chem


In [44]:
df.cost.dtype

dtype('O')

In [22]:
# Strip out '£' and '$' signs and convert cost to numeric values.

df['cost'] = (df['cost'].str.strip('£'))

In [73]:
for i in df.cost:
    if df.cost.str.contains('$'):
        df.cost = df.cost.replace('$','')
        df.cost = df.cost.astype('float64') 
        df.cost = df.cost * 0.71

# np.where
# np.replace

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [50]:
df.loc[df.journal_title == 'BMC Genomics']

,PMID_PMCID,publisher,journal_title,Article_title,cost,journal_lower
181,PMC2843621,BioMed Central,BMC Genomics,Trichomonas vaginalis vast BspA-like gene fami...,1204.38$,bmc genomics
182,PMCID:\n PMC3636053,BioMed Central,BMC Genomics,Enhancing the utility of Proteomics Signature ...,1254.6$,bmc genomics
183,3526451,Biomed Central,BMC Genomics,Advances in genome-wide RNAi cellular screens:...,1476$,bmc genomics
226,3681581,BioMed Central,BMC Genomics,Transcriptional adaptation of pneumococci and ...,608.52,bmc genomics


In [31]:
# This didn't work - print(df[df['cost'].str.contains('$')]['cost'])

0          0.00
1       2381.04
2        642.56
3        669.64
4        685.88
5       2392.20
6       2367.95
7        649.33
8       1294.59
9       1294.78
10      1329.69
11      1287.20
12      1324.66
13       665.64
14      1006.72
15       238.08
16       265.67
17      2035.00
18      3108.08
19      1267.76
20      2286.73
21       947.07
22      1186.80
23       642.89
24      1533.29
25      2082.74
26      2314.10
27       759.67
28      1565.26
29       572.74
         ...   
2097    2236.02
2098    2275.02
2099    2373.57
2100     791.19
2101    1551.49
2102    1961.68
2103    1982.35
2104    2239.22
2105    2377.65
2106    2272.15
2107    1900.70
2108    3026.17
2109    1851.15
2110    1812.89
2111    1816.70
2112    2168.05
2113    1409.96
2114    2400.00
2115    1968.63
2116    2015.72
2117    1836.92
2118    1836.92
2119    1375.80
2120    1836.92
2121    2009.65
2122    1334.15
2123    1834.77
2124    1834.77
2125    2374.52
2126    2034.75
Name: cost, Length: 2127

In [24]:
print(df.cost)

0          0.00
1       2381.04
2        642.56
3        669.64
4        685.88
5       2392.20
6       2367.95
7        649.33
8       1294.59
9       1294.78
10      1329.69
11      1287.20
12      1324.66
13       665.64
14      1006.72
15       238.08
16       265.67
17      2035.00
18      3108.08
19      1267.76
20      2286.73
21       947.07
22      1186.80
23       642.89
24      1533.29
25      2082.74
26      2314.10
27       759.67
28      1565.26
29       572.74
         ...   
2097    2236.02
2098    2275.02
2099    2373.57
2100     791.19
2101    1551.49
2102    1961.68
2103    1982.35
2104    2239.22
2105    2377.65
2106    2272.15
2107    1900.70
2108    3026.17
2109    1851.15
2110    1812.89
2111    1816.70
2112    2168.05
2113    1409.96
2114    2400.00
2115    1968.63
2116    2015.72
2117    1836.92
2118    1836.92
2119    1375.80
2120    1836.92
2121    2009.65
2122    1334.15
2123    1834.77
2124    1834.77
2125    2374.52
2126    2034.75
Name: cost, Length: 2127

In [25]:
df.cost.dtype

dtype('O')

In [26]:
df['cost']  = df['cost'].astype('float64') 

ValueError: could not convert string to float: '1600.25$'

In [ ]:
df.dtypes

In [ ]:
df['journal_lower'].nunique()

### Here are the five most common journals and the total articles for each.

In [ ]:
five = df.groupby(['journal_lower']).count()
five['journal_title'].sort_values(ascending=False).head(5)

### 2. Calculate the mean, median, and standard deviation of the open-access cost per article for each journal 

In [ ]:
topfive = df.groupby(['journal_lower'])['cost'].agg(['count', 'mean', 'median','std']).reset_index()
topfive_sorted = topfive.sort_values(['count'], ascending=False)
topfive_sorted.head()

In [ ]:
print(df.cost.max())
print(df.cost.max().dtype)

In [ ]:
df[df['cost'] > 3999]['cost'] = np.nan

In [ ]:
df['cost'] > 1000

In [ ]:
df[df['cost'] > 1000]